# **Tutoriel** - Fine-tuning de Llama

<img src="https://raw.githubusercontent.com/opinionscience/FabriqueLLM/main/illustration/llama_image.png" alt="Llama logo"  width="500"/>

Ce carnet de code permet d'effectuer le fine-tuning d'un grand modèle de langue développé par Meta, Llama. Tout en n'étant pas diffusé en open source mais seulement à des fins de recherche non-commercial, Llama a eu un impact significatif sur le développement de LLMs ouverts alternatifs à chatGPT. Une grande partie des méthodes et des jeux de données utilisés pour adapter les LLMs ont été d'abord pensés pour LLama.

LLama est disponible sous quatre versions à 7B, 13B, 33B and 65B : le B correspond ici à un milliard de paramètres. Nous allons ici utiliser la plus plus petite version (7B) qui est compatible avec la version gratuite de Google Colab.

Ce carnet de code est basé sur une version modifiée de LLMTune. LLMTune est un projet de recherche de Cornell Tech et de Cornell University (Cornell University). L'exécution de la version originale sur Google Colab donne lieu à plusieurs bugs que j'ai corrigés.

Cette démonstration ne fait tourner qu'une seule *epoch* ce qui est suffisant pour avoir un premier aperçu. Pour obtenir un bon modèle, il est conseillé de faire tourner le fine-tuning pendant trois *epochs*. Sur notre corpus de démonstration de 2000 instructions une *epoch* prendra environ 30 minutes.

# Installation

En tout premier lieu nous vérifions si nous disposons de suffisamment de mémoire vive (au moins 24go) sinon ce n'est pas la peine de lancer le script.

In [ ]:
!nvidia-smi

Sun Jun 11 14:57:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    40W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Une telle configuration est suffisante pour le fine-tuning de llama-7b-4bit basé sur LLMTune (mais pas pour llama-13b-4bit, sans même parler des versions suivantes)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/llama"

Mounted at /content/drive
/content/drive/My Drive/llama


Nous allons utiliser llmtune. C'est une petite application python disponible sur Github qui permet d'effectuer le fine-tuning de Llama (pour l'instant depuis mon fork comme il y a un bug sur la version de base)

In [ ]:
!git clone https://github.com/Pclanglais/llmtune.git

Cloning into 'llmtune'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 150 (delta 58), reused 36 (delta 29), pack-reused 54
Receiving objects: 100% (150/150), 48.26 KiB | 2.54 MiB/s, done.
Resolving deltas: 100% (69/69), done.


Nous installons également les dépendances.

In [ ]:
%cd llmtune
!pip install -r requirements.txt
!python setup.py install

/content/drive/My Drive/llama/llmtune
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Cloning https://github.com/huggingface/transformers (to revision 9417c924af539be5f941c8a709a96b60dfe29eb3) to /tmp/pip-install-_plc44yu/transformers_c9427feddaa74835b13dc5c1d324260d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-install-_plc44yu/transformers_c9427feddaa74835b13dc5c1d324260d
  Running command git rev-parse -q --verify 'sha^9417c924af539be5f941c8a709a96b60dfe29eb3'
  Running command git fetch -q https://github.com/huggingface/transformers 9417c924af539be5f941c8a709a96b60dfe29eb3
  Running command git checkout -q 9417c924af539be5f941c8a709a96b60dfe29eb3
  Resolved https://github.com/huggingface/transformers to commit 9417c924af539be5f941c8a709a96b60dfe29eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Enfin nous récupérons le modèle de base de Llama

In [ ]:
!wget https://huggingface.co/decapoda-research/llama-7b-hf-int4/resolve/main/llama-7b-4bit.pt

--2023-07-11 09:16:04--  https://huggingface.co/decapoda-research/llama-7b-hf-int4/resolve/main/llama-7b-4bit.pt
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.88, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/98/d7/98d7c1a709a7ae2b2b2dbbc8fa82286eae5bf3bff3f004e7d5843c4344e64b11/b48471adcc7e20542f9cacc348725b4ad36c3321ca2015bbd57d3876302426ee?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-7b-4bit.pt%3B+filename%3D%22llama-7b-4bit.pt%22%3B&Expires=1689326164&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTMyNjE2NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85OC9kNy85OGQ3YzFhNzA5YTdhZTJiMmIyZGJiYzhmYTgyMjg2ZWFlNWJmM2JmZjNmMDA0ZTdkNTg0M2M0MzQ0ZTY0YjExL2I0ODQ3MWFkY2M3ZTIwNTQyZjljYWNjMzQ4NzI1YjRhZDM2YzMzMjFjYTIwMTViYmQ1N2QzODc

## Le corpus d'instructions

Nous récupérons un set d'instruction. Dans l'optique d'un simple exemple de démonstration nous utilisons ici une sélection aléatoire de différents corpus d'instructions traduits en français par le projet Vicogne.

In [ ]:
!wget https://raw.githubusercontent.com/opinionscience/EUInstruct/main/sample/french_novel17_instruction_simple.json

--2023-07-11 10:17:26--  https://raw.githubusercontent.com/opinionscience/EUInstruct/main/sample/french_novel17_instruction_simple.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2336881 (2.2M) [text/plain]
Saving to: ‘french_novel17_instruction_simple.json’

french_novel17_inst 100%[===================>]   2.23M  --.-KB/s    in 0.04s   

2023-07-11 10:17:26 (53.9 MB/s) - ‘french_novel17_instruction_simple.json’ saved [2336881/2336881]



Ces instructions utilisent le format classique du projet Alpaca de Stanford : *instructions*, *input* (optionnellement) et *output*. En résumé, les instructions correspondent à des exemples de prompts que pourraient laisser les utilisateurs du LLM, les *outputs* à la réponse que le LLM devrait générer et les *inputs* apporte des éléments de contextes supplémentaires (par exemple sous la forme de textes cités en exemple)

In [ ]:
import json

with open('french_novel17_instruction_simple.json', 'r') as f:
    data = json.load(f)

json_formatted_str = json.dumps(data[0:3], indent=2)

print(json_formatted_str)

[
  {
    "instruction": "\u00c9crivez un texte qui d\u00e9peint un affrontement verbal entre Lao damas et Eurialus, o\u00f9 ce dernier insulte Ulysse en le traitant de marin incomp\u00e9tent. Le style utilis\u00e9 doit \u00eatre soutenu et empreint d'une certaine gravit\u00e9.",
    "input": "",
    "output": "Si Lao damas fut content de cette mode\u017fte repon\u017feil n'en fut pas de m\u00eame de quelques -uns des autres qui \u017fe per\u017fuadant que s'il \u017fe defen doit d'entrer en licecela ne provenoit que de \u017fa foiblefle & de fon peu d'experience dans ces \u017fortes d'exercices \u017fe mirent \u00e0 le prefler d'u ne maniere fort incivile . Eurialus particuliere ment tout boufi de l'honneur qu'il venoit d'ac querir piqua Uli\u017fle pardes paroles non leule. ment dures mais injurieu\u017feslc taxant d'e\u017ftre plus propre \u00e0 courir la mer \u017fur quelque vailleau CON"
  },
  {
    "instruction": "\u00c9crivez un texte litt\u00e9raire historique dans le style du

# Finetuning du modèle

Tout est prêt à lancer le fine-tuning du modèle. Nous allons juste créer le dossier d'accueil des fichiers de fine-tuning

In [ ]:
!mkdir llama-7b-novel217

Et nous sommes prêt à lancer la grande commande. Il y a beaucoup de paramètre mais seulement quelqu'uns sont importants :
* Nous allons utiliser le modèle Llama-7b de base et leurs poids correspondants (llama-7b-4bit)
* Le fine-tuning sera effectué sur le set d'instruction *eu_translate_fr_sample.json* (évidemment à changer si vous optez pour un autre jeu de données).
* Les fichiers du modèle seront placés dans le dossier *llama-7b-sample* (de nouveau à changer pour le nom de votre modèle).
* Nous ne ferons tourner le fine-tuning que sur une *epoch* ce qui est suffisant pour un premier test.

Après avoir lancé le script, Google Colab va tourner pendant un peu moins de 40 minutes.

Si tout se passe bien vous verrez défiler le processus d'entraînement avec trois indicateurs régulièrement réactualisés : "{'loss': 1.8581, 'learning_rate': 0.0002993736951983298, 'epoch': 0.0}" :
* Le "loss" c'est en quelque sorte le taux d'erreur du modèle : plus cette mesure est basse et plus le modèle parvient à prédire des textes assez approchants de ceux qui sont présent dans le corpus d'instruction.
* Le *learning rate* (taux d'apprentissage) c'est la capacité du modèle à mémoriser de nouveaux éléments mais aussi à en oublier des anciens. Cet indicateur va constamment baisser au fur et à mesure de l'apprentissage.
* L'*epoch* c'est le cycle d'apprentissage. Comme nous n'avons défini qu'une *epoch* cela correspondra à des pourcentages (de 0 à 0.99 à la fin de l'entraînement).

In [ ]:
!llmtune finetune \
    --model llama-7b-4bit \
    --lr=3e-4 \
    --epochs 3 \
    --save_total_limit 3 \
    --save_steps 500 \
    --weights llama-7b-4bit.pt \
    --adapter llama-7b-novel217 \
    --mbatch_size=1 \
    --batch_size=2 \
    --epochs=3 \
    --lora_r=8 \
    --lora_alpha=16 \
    --lora_dropout=0.05 \
    --warmup_steps=5 \
    --dataset french_novel17_instruction_simple.json

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
trainable params: 4194304 || all params: 266604544 || trainable%: 1.5732304997772282
Found cached dataset json (/root/.cache/huggingface/datasets/json/default-768964e92ee08ad9/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100% 1/1 [00:00<00:00, 506.50it/s]
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/json/default-768964e92ee08ad9/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-a7910bc70c173c6d.arrow and /root/.cache/huggingface/datasets/json/default-768964e92ee08ad9/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-f3067803ad173f5a.arrow
2023-07-11 10:25:37.751295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not f

In [ ]:
!ls

build				   llama-7b-french-novel  README.md
dist				   llmtune		  requirements.txt
instruction_french_novel.json	   llmtune.egg-info	  setup.py
instruction_french_novel.json.zip  __MACOSX
llama-7b-4bit.pt		   quant_cuda.egg-info


Si tout se passe bien, vous devrez avoir comlètement fini l'entraînement.

Après un petit temps de synchronisation entre Google Colab, vous allez voir apparaître deux fichiers dans le dossier du modèle de fine-tuning : adapter_model.bin (le modèle proprepement dit) et adapter_model.config (un fichier de configuration). À noter que le modèle de fine-tuning est considérablement plus petit que le modèle d'origine : c'est en quelque sorte un modèle complémentaire qui vient ajuster le LLM (et il en aura toujours besoin pour fonctionner).

# Générer du texte

Et maintenant il est possible de générer du texte. La fonction par défaut de llmtune n'est pas pour l'instant pas très pratique mais cela devrait s'améliorer prochainement.

In [ ]:
!llmtune generate \
    --model llama-7b-4bit \
    --weights llama-7b-4bit.pt \
    --adapter llama-7b-novel217 \
    --max-length 400 \
    --min-length 300 \
    --instruction "Écrivez un texte littéraire dans le style du XVIIe siècle décrivant un peuple de lamas intelligents au Pérou. Vous pouvez notamment évoquer les mœurs et coutumes de ces animaux très savants. ###OUTPUT"

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
llama-7b-novel217 loaded
2023-07-11 11:29:33.244721: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Les Lamas des provinces peruanes ſont d'une finance inſtuſe & d'une mœurs des plus civilizées: Ils ont des habitudes qui prê. ont les femmes plus de ſerené qu'elles ne le font à terre : La majorité de ces peuples ſont catholiques & leur pallieu leſon ne doit jamais auoir été troublé par les religions novembres ; il eſt de ces choſes qui n'ont été jamais in. convenables pour l'Amérique. Ils ont des tetres & des langages perfeitement differenfles</s>AXI i & du Roi de Naples auſſi bien qu'au Roi de France. Les plus fins de ce peuple qui demeurent à Parigo. ont une civilité & une maniere de porter la perruque qui eſt 